<a href="https://colab.research.google.com/github/xaesalvaje/me.ai/blob/main/create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade jupyter_http_over_ws>=0.0.7 && \
  jupyter serverextension enable --py jupyter_http_over_ws

Enabling: jupyter_http_over_ws
- Writing config: /root/.jupyter
    - Validating...
      jupyter_http_over_ws 0.0.7 OK


In [2]:
!pip install transformers torch accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.2 MB/s eta 0:00:00


In [6]:
from transformers import pipeline, set_seed
import random

In [7]:
set_seed(42)

In [8]:
with open("mywords.txt", "r") as f:
    texts = f.read().splitlines()

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import TrainerCallback
from torch.nn.utils import clip_grad_norm_
from transformers.utils.versions import require_version
from accelerate import Accelerator

require_version("accelerate", "0.3.0")

class MixedPrecisionCallback(TrainerCallback):
    def __init__(self):
        self.scaler = None
        
    def on_init_end(self, args, state, control, **kwargs):
        self.scaler = Accelerator().scaler
    
    def on_before_step(self, args, state, control, **kwargs):
        if self.scaler is None:
            return
        self.scaler.step(state.optimizer)
        self.scaler.update()
        
    def on_after_backward(self, args, state, control, **kwargs):
        if self.scaler is None:
            return
        clip_grad_norm_(self.scaler.master_params(state.optimizer), state.args.max_grad_norm)

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

train_dataset = TextDataset(tokenizer=tokenizer, 
                            file_path="mywords.txt", 
                            block_size=128)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                mlm=False)

training_args = TrainingArguments(
    output_dir="./models",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,  # Decreased batch size
    gradient_accumulation_steps=32,  # Increase effective batch size
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    callbacks=[MixedPrecisionCallback()]
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (86719 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=21, training_loss=4.753547668457031, metrics={'train_runtime': 4991.9042, 'train_samples_per_second': 0.136, 'train_steps_per_second': 0.004, 'total_flos': 156021716680704.0, 'train_loss': 4.753547668457031, 'epoch': 0.99})

In [11]:
generator = pipeline('text-generation', model=model, tokenizer='microsoft/DialoGPT-medium')

def generate_response(prompt, length=50):
    response = generator(prompt, max_length=length, do_sample=True, temperature=0.7)[0]['generated_text']
    return response.strip()

In [13]:
prompt = "Hi, how are you?"
response = generate_response(prompt)
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hi, how are you?


In [14]:
# Save the model
model.save_pretrained("models/model")
tokenizer.save_pretrained("models/model")

('models/model/tokenizer_config.json',
 'models/model/special_tokens_map.json',
 'models/model/vocab.json',
 'models/model/merges.txt',
 'models/model/added_tokens.json',
 'models/model/tokenizer.json')